<a href="https://colab.research.google.com/github/devpandey2010/NLP-NATURAL-LANGUAGE-PROCESSOR-/blob/main/Time_Series_Forecasting_with_LSTM_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📦 Dataset:Use the Daily Minimum Temperatures dataset from Melbourne, Australia.


Build an LSTM model that learns to predict the temperature of the next day given the past 7 days of temperatures.

In [ ]:
#Download the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
df=pd.read_csv('https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv')
df
df_copy=df.copy()


In [ ]:
#lets extraxt the date month and year separetelly
df_copy['year']=df_copy['Date'].apply(lambda x:x.split('-')[0])
df_copy['month']=df_copy['Date'].apply(lambda x:x.split('-')[1])
df_copy['day']=df_copy['Date'].apply(lambda x:x.split('-')[2])
df_copy

#or


,Date,Temp,year,month,day
0,1981-01-01,20.7,1981,01,01
1,1981-01-02,17.9,1981,01,02
2,1981-01-03,18.8,1981,01,03
3,1981-01-04,14.6,1981,01,04
4,1981-01-05,15.8,1981,01,05
...,...,...,...,...,...
3645,1990-12-27,14.0,1990,12,27
3646,1990-12-28,13.6,1990,12,28
3647,1990-12-29,13.5,1990,12,29
3648,1990-12-30,15.7,1990,12,30


In [ ]:

# df_copy['date']=pd.to_datetime(df_copy['Date'])
# df_copy['year']=df_copy['date'].dt.year
# df_copy['month']=df_copy['date'].dt.month
# df_copy['day']=df_copy['date'].dt.day
# df_copy

,Date,Temp,year,month,day,date
0,1981-01-01,20.7,1981,1,1,1981-01-01
1,1981-01-02,17.9,1981,1,2,1981-01-02
2,1981-01-03,18.8,1981,1,3,1981-01-03
3,1981-01-04,14.6,1981,1,4,1981-01-04
4,1981-01-05,15.8,1981,1,5,1981-01-05
...,...,...,...,...,...,...
3645,1990-12-27,14.0,1990,12,27,1990-12-27
3646,1990-12-28,13.6,1990,12,28,1990-12-28
3647,1990-12-29,13.5,1990,12,29,1990-12-29
3648,1990-12-30,15.7,1990,12,30,1990-12-30


In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df_copy['Temp']=scaler.fit_transform(df_copy[['Temp']])



In [ ]:
df_copy

,Date,Temp,year,month,day
0,1981-01-01,0.787072,1981,01,01
1,1981-01-02,0.680608,1981,01,02
2,1981-01-03,0.714829,1981,01,03
3,1981-01-04,0.555133,1981,01,04
4,1981-01-05,0.600760,1981,01,05
...,...,...,...,...,...
3645,1990-12-27,0.532319,1990,12,27
3646,1990-12-28,0.517110,1990,12,28
3647,1990-12-29,0.513308,1990,12,29
3648,1990-12-30,0.596958,1990,12,30


In [ ]:
def create_dataset(data,look_back=7):
  x,y=[],[]
  for i in range(len(data)-look_back):
    x.append(data[i:i+look_back])
    y.append(data[i+look_back])
  return np.array(x),np.array(y)

data=df["Temp"].values
x,y=create_dataset(data)

In [ ]:
#LSTM expects shape: (samples, timesteps, features)
x = x.reshape((x.shape[0], x.shape[1], 1))

In [ ]:
x.shape

(3643, 7, 1)

In [ ]:
split = int(len(X) * 0.8)
X_train, X_test = x[:split], x[split:]
y_train, y_test = y[:split], y[split:]

In [ ]:
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,LSTM
model=Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(X_train.shape[1], 1)))
# Second LSTM layer (also returns sequences)
model.add(LSTM(32, return_sequences=True))
# Third LSTM layer (last one, returns a single output vector)
model.add(LSTM(16, return_sequences=False))
# Final Dense layer to predict 1 value
model.add(Dense(1))
model.compile(optimizer='adam',loss='mse',metrics=['r2_score'])
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_38 (LSTM)                  │ (None, 7, 64)          │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_39 (LSTM)                  │ (None, 7, 32)          │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_40 (LSTM)                  │ (None, 16)             │         3,136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 32,465 (126.82 KB)

 Trainable params: 32,465 (126.82 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test))

Epoch 1/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 5s 14ms/step - loss: 104.2959 - r2_score: -5.6232 - val_loss: 53.9348 - val_r2_score: -2.2003
Epoch 2/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - loss: 43.5638 - r2_score: -1.7202 - val_loss: 37.1983 - val_r2_score: -1.2072
Epoch 3/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - loss: 33.4264 - r2_score: -0.9642 - val_loss: 27.9752 - val_r2_score: -0.6599
Epoch 4/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 22.9906 - r2_score: -0.4109 - val_loss: 22.6284 - val_r2_score: -0.3427
Epoch 5/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 20.5532 - r2_score: -0.2101 - val_loss: 19.7974 - val_r2_score: -0.1747
Epoch 6/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - loss: 17.9314 - r2_score: -0.0915 - val_loss: 18.3131 - val_r2_score: -0.0866
Epoch 7/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - loss: 17.5072 - r2_score: -0.0279 - val_loss: 17.5974 - val_r2_score: -0.0442
Epoch 8/20
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - loss: 16.5491 - r2_score: -0.

In [ ]:
predictions = model.predict(X_test)
predictions

23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([[13.225247 ],
       [14.252227 ],
       [14.532684 ],
       [14.442806 ],
       [12.109031 ],
       [10.966711 ],
       [12.467158 ],
       [13.799432 ],
       [13.856658 ],
       [12.104619 ],
       [12.013911 ],
       [13.073234 ],
       [11.578996 ],
       [13.00213  ],
       [14.189036 ],
       [14.499782 ],
       [14.129346 ],
       [13.162432 ],
       [13.044188 ],
       [14.147151 ],
       [13.295779 ],
       [14.052047 ],
       [12.306572 ],
       [12.453738 ],
       [13.814567 ],
       [14.095359 ],
       [14.371808 ],
       [14.600024 ],
       [14.635017 ],
       [14.4465685],
       [14.562127 ],
       [14.559821 ],
       [14.031386 ],
       [14.4756565],
       [14.332168 ],
       [13.554881 ],
       [13.038564 ],
       [14.175794 ],
       [14.491294 ],
       [14.408231 ],
       [14.19922  ],
       [14.401559 ],
       [14.175809 ],
       [14.558131 ],
       [14.61015  ],
       [14.639363 ],
       [14.644599 ],
       [14.64